In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import tree
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import catboost as cb
import time

In [33]:
# загружаем датасет
data = load_iris(as_frame=True)
predictors = data.data
target = data.target

Баггинг

In [34]:
# Разделим исходный датасет на обучающую и тестовую выборки 
A_train, A_test, y_train, y_test = train_test_split(predictors,
                                                    target,
                                                    train_size=0.8)



In [35]:
# обучим случайный лес
random_forest = RandomForestClassifier(max_depth=15, min_samples_split=10).fit(
    A_train, y_train)


In [36]:
y_preds_d = random_forest.predict(A_train)
print('F1 мера для тренировочных данных', f1_score(y_preds_d, y_train, average='macro'))

F1 мера для тренировочных данных 0.9823232323232324


In [37]:
y_preds = random_forest.predict(A_test)
print('F1 мера для тестовых данных', f1_score(y_preds, y_test, average='macro'))

F1 мера для тестовых данных 0.9315476190476191


In [38]:
# Проблема заключается в том, что эти параметры нужно каким-то образом выбрать. 
# Можно провести перебор параметров и для каждой их комбинации обучить модели, затем выбрать лучшую. 

random_forest = RandomForestClassifier()

params_grid = {
    'max_depth': [12, 18],
    'min_samples_leaf': [3, 10],
    'min_samples_split': [6, 12]
}
start_time = time.time()
grid_search_random_forest = GridSearchCV(estimator=random_forest,
                                         param_grid=params_grid,
                                         scoring='f1_macro',
                                         cv=4)


In [39]:
grid_search_random_forest.fit(A_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [12, 18], 'min_samples_leaf': [3, 10],
                         'min_samples_split': [6, 12]},
             scoring='f1_macro')

In [40]:
best_model = grid_search_random_forest.best_estimator_
best_model
time1 = time.time() - start_time

In [41]:
y_preds_d = best_model.predict(A_train)
print('F1 мера для тренировочных данных', f1_score(y_preds_d, y_train, average='macro'))

F1 мера для тренировочных данных 0.9646464646464646


In [42]:
y_preds = best_model.predict(A_test)
score1 = f1_score(y_preds, y_test, average='macro')
print('F1 мера для тестовых данных', score1)

F1 мера для тестовых данных 0.9010752688172042


Бустинг

In [43]:
start_time = time.time()
model_catboost_clf = cb.CatBoostClassifier(iterations=3000,
                                           task_type='GPU',
                                           devices='0')
model_catboost_clf.fit(A_train, y_train)
time2 = time.time() - start_time

Learning rate set to 0.016865
0:	learn: 1.0749088	total: 18.4ms	remaining: 55.2s
1:	learn: 1.0520140	total: 28ms	remaining: 42s
2:	learn: 1.0318189	total: 40.4ms	remaining: 40.3s
3:	learn: 1.0120202	total: 55.7ms	remaining: 41.7s
4:	learn: 0.9928024	total: 67.2ms	remaining: 40.2s
5:	learn: 0.9741409	total: 81.1ms	remaining: 40.5s
6:	learn: 0.9560848	total: 94ms	remaining: 40.2s
7:	learn: 0.9369467	total: 105ms	remaining: 39.2s
8:	learn: 0.9184320	total: 116ms	remaining: 38.7s
9:	learn: 0.9018344	total: 128ms	remaining: 38.3s
10:	learn: 0.8857958	total: 138ms	remaining: 37.5s
11:	learn: 0.8701894	total: 151ms	remaining: 37.7s
12:	learn: 0.8535934	total: 162ms	remaining: 37.2s
13:	learn: 0.8375287	total: 172ms	remaining: 36.6s
14:	learn: 0.8230872	total: 184ms	remaining: 36.7s
15:	learn: 0.8077741	total: 201ms	remaining: 37.5s
16:	learn: 0.7940759	total: 217ms	remaining: 38s
17:	learn: 0.7807199	total: 228ms	remaining: 37.7s
18:	learn: 0.7665617	total: 246ms	remaining: 38.6s
19:	learn: 0

In [44]:
y_preds_t = model_catboost_clf.predict(A_train, task_type='CPU')
print('F1 мера для тренировочных данных', f1_score(y_preds_t, y_train, average='macro'))

F1 мера для тренировочных данных 1.0


In [45]:
y_preds = model_catboost_clf.predict(A_test, task_type='CPU')
score2 = f1_score(y_preds, y_test, average='macro')
print('F1 мера для тестовых данных', score2)

F1 мера для тестовых данных 0.9315476190476191


Сравнить результаты работы алгоритмов (время работы и качество моделей). 

In [46]:
df = pd.DataFrame({
    'Algorithm': ['Баггинг', 'Бустинг'],
    'time': [time1, time2],
    'Efficiency': [score1, score2]
})

print(df)

  Algorithm       time  Efficiency
0   Баггинг   4.013005    0.901075
1   Бустинг  29.348035    0.931548
